In [2]:
import os
os.environ["GOOGLE_API_KEY"] = "Aaaaaaaaaaaaa"

from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("google_genai:gemini-2.5-flash-lite")

system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("What is Data Science")

# Use with chat models
messages = [system_msg, human_msg]
response = model.invoke(messages)  # Returns AIMessage
print(response.content)
# This response is without streaming

Data science is a **multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data.** It's about turning raw data into actionable information that can drive better decision-making and solve complex problems.

Think of it as a blend of several disciplines, including:

*   **Statistics and Probability:** To understand the underlying patterns, variability, and relationships within data, and to make inferences and predictions.
*   **Computer Science and Programming:** To collect, clean, store, process, and analyze large datasets efficiently. Languages like Python and R are fundamental.
*   **Mathematics:** For developing and understanding algorithms, models, and statistical techniques.
*   **Domain Expertise:** Understanding the specific industry or area the data comes from is crucial for interpreting results and asking the right questions. For example, a data scientist working in healthcare w

In [3]:
system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("What is Machine learning")

# Use with chat models
messages = [system_msg, human_msg]
chunks = []
full_message = None
for chunk in model.stream(messages):
    chunks.append(chunk)
    print(chunk.text)
    full_message = chunk if full_message is None else full_message + chunk
# This response is with streaming - way 1

Machine learning is a sub
field of artificial intelligence (AI) that focuses on building systems that can **learn from
 data** and make predictions or decisions without being explicitly programmed.

Think of it like teaching a child. Instead of giving them a rigid set of rules for every single situation, you show them many examples and let them figure out the patterns and rules
 on their own. Machine learning algorithms do something similar with data.

Here's a breakdown of the key concepts:

**1. Learning from Data:**

*   **Data is the fuel:** Machine learning models require large amounts of data to
 learn effectively. This data can be in various forms: numbers, text, images, audio, video, etc.
*   **Identifying patterns:** The algorithms analyze this data to discover hidden patterns, relationships, and trends.
*   **Building
 a model:** Based on these patterns, the algorithm creates a "model" – essentially a mathematical representation that can be used to make predictions or classif

# Supported stream modes
    Pass one or more of the following stream modes as a list to the stream or astream methods:
    Mode	    Description
   1. updates:	    Streams state updates after each agent step. If multiple updates are made in the same step (e.g., multiple nodes are run), those updates are streamed separately.
   2. messages:	Streams tuples of (token, metadata) from any graph nodes where an LLM is invoked.
3. custom:	    Streams custom data from inside your graph nodes using the stream writer.

# 1.Agent progress : Updates mode

In [19]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)
for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'id': 'ba3913ff-8d95-40f7-88ff-eaed74f23455', 'name': 'get_weather', 'args': {'city': 'SF'}}]
step: tools
content: [{'type': 'text', 'text': "It's always sunny in SF!"}]
step: model
content: [{'type': 'text', 'text': "The weather in SF is: It's always sunny in SF!"}]


In [21]:
from langchain.agents import create_agent
agent = create_agent(model=model)
for chunk in agent.stream({"messages": [{"role": "user", "content": "What is the Data Science?"}]},
                            stream_mode="updates",):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'text', 'text': 'Data Science is a **multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data.** It\'s about understanding data, finding patterns, and using those patterns to make informed decisions, predict future outcomes, and build intelligent systems.\n\nThink of it as a combination of several powerful disciplines, all working together to make sense of the ever-growing ocean of data we generate.\n\nHere\'s a breakdown of the key components and what makes Data Science so important:\n\n**Core Disciplines Involved:**\n\n*   **Mathematics and Statistics:** This forms the bedrock of data science. Understanding probability, statistical modeling, hypothesis testing, linear algebra, and calculus is crucial for analyzing data, building models, and interpreting results.\n*   **Computer Science:** This provides the tools and techniques for handling large datas

# 2. LLm tokens - messages mode

In [23]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)
for token, metadata in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

node: model
content: [{'type': 'tool_call_chunk', 'id': '0e604717-2401-4692-85c7-359e73562dfb', 'name': 'get_weather', 'args': '{"city": "SF"}'}]


node: model
content: []


node: tools
content: [{'type': 'text', 'text': "It's always sunny in SF!"}]


node: model
content: [{'type': 'text', 'text': 'The weather in SF is:'}]


node: model
content: [{'type': 'text', 'text': " It's always sunny in SF!"}]


node: model
content: []




In [24]:
from langchain.agents import create_agent
agent = create_agent(model=model)
for token, metadata in agent.stream({"messages": [{"role": "user", "content": "What is the Data Science?"}]},
                            stream_mode="messages",):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

node: model
content: [{'type': 'text', 'text': 'Data Science is a **'}]


node: model
content: [{'type': 'text', 'text': 'multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge'}]


node: model
content: [{'type': 'text', 'text': " and insights from structured and unstructured data.** It's essentially about understanding and leveraging data to make better decisions, solve complex problems, and create new products or services.\n\nThink of it as a blend of several disciplines, all working together to unlock"}]


node: model
content: [{'type': 'text', 'text': ' the power of data:\n\n**Key Components of Data Science:**\n\n*   **Statistics and Probability:** This forms the bedrock of data science. It provides the tools to understand data distributions, test hypotheses, quantify uncertainty, and make inferences.\n'}]


node: model
content: [{'type': 'text', 'text': '*   **Computer Science and Programming:** Data scientists need to be prof

# 3. Custom updates mode

In [33]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer  


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()  
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="custom"
):
    print(chunk)

Looking up data for city: SF
Acquired data for city: SF
